In [1]:
def make_uk_text():
    import gzip
    import json
    
    fname = 'jawiki-country.json'
    
    with open(fname) as file:
        for line in file:
            article = json.loads(line)
            if article['title'] == 'イギリス':
                uk_text = article['text']
                break
    return uk_text

In [2]:
def remove_markup(text):
    import re
    
    pattern = re.compile(r'''
    ('{2,5})
    (.*?)
    \1
    ''', re.MULTILINE + re.VERBOSE)
    
    pattern2 = re.compile(r'''
    (?:(?<!\#REDIRECT)|(?<!\[\[Category))
    \[\[
    (?:[^|]*?\|)??
    ([^|]*?)
    \]\]
    ''', re.MULTILINE + re.VERBOSE)
    
    text2 =  pattern2.sub(r'\1', pattern.sub(r'\2', text))
    
    pattern3 = re.compile(r'\[\[(.*?)\]\]')
    
    pattern4 = re.compile(r'\[http:\/\/(.*?)\]')
    
    pattern5 = re.compile(r'\{\{(.*?)\}\}')
    
    pattern6 = re.compile(r'<(.*?)>')
    
    return  pattern6.sub('', pattern5.sub('', pattern4.sub('', pattern3.sub('', text2))))

In [3]:
import re

fields_dic = {}

pattern = re.compile(r'''
    \{\{基礎情報.*?$
    (.*?)
    ^\}\}
    ''', re.MULTILINE + re.VERBOSE + re.DOTALL)

pattern2 = re.compile(r'''
    ^
    \|
    (.*?)
    \s*
    =
    \s*
    (.*?)
    (?:(?=\n\|)|(?=\n$))
    ''', re.MULTILINE + re.VERBOSE)

BasicInformations = pattern.findall(make_uk_text())
fields = pattern2.findall(BasicInformations[0])
for field in fields:
    fields_dic[field[0]] = remove_markup(field[1])

In [4]:
import urllib.parse, urllib.request
import json

fname_flag = fields_dic['国旗画像']

url = 'https://www.mediawiki.org/w/api.php?' \
    + 'action=query' \
    + '&titles=File:' + urllib.parse.quote(fname_flag) \
    + '&format=json' \
    + '&prop=imageinfo' \
    + '&iiprop=url'

request = urllib.request.Request(url, headers={'User-Agent': 'kadotani_NLP'})
response = urllib.request.urlopen(request)
response_data = json.loads(response.read().decode())

In [5]:
print(json.dumps(response_data, indent=2))

{
  "continue": {
    "iistart": "2019-09-10T16:52:58Z",
    "continue": "||"
  },
  "query": {
    "pages": {
      "-1": {
        "ns": 6,
        "title": "File:Flag of the United Kingdom.svg",
        "missing": "",
        "known": "",
        "imagerepository": "shared",
        "imageinfo": [
          {
            "url": "https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg",
            "descriptionurl": "https://commons.wikimedia.org/wiki/File:Flag_of_the_United_Kingdom.svg",
            "descriptionshorturl": "https://commons.wikimedia.org/w/index.php?curid=347935"
          }
        ]
      }
    }
  }
}


In [6]:
result = response_data['query']['pages']['-1']['imageinfo'][0]['url']
print(result)

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
